# Scraping Michelin Guide for Restaraunts

In [2]:
# Imports
from bs4 import BeautifulSoup
import time
from IPython.core.display import clear_output
import random
from requests import get
from warnings import warn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

# Setting base url
url = 'https://guide.michelin.com/en/restaurants/1-star-michelin/page/1'

C:\Users\shaun\AppData\Local\Temp\ipykernel_33384\464270697.py:4: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [3]:
page = get(url)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find(class_ = 'search-results__column col-lg-12')
restaurants = results.find_all(class_ = 'col-md-6 col-lg-4 col-xl-3')
print(restaurants[0])
# <div class="col-md-6 col-lg-4 col-xl-3">

<div class="col-md-6 col-lg-4 col-xl-3">
<div class="card__menu selection-card box-placeholder js-restaurant__list_item js-match-height js-map" data-id="103865" data-index="0" data-lat="49.2876672" data-lng="-123.1191909" data-map-pin-name="ONE_STAR" data-view="restaurant">
<div class="card__menu-image">
<div class="card__menu-image--top">
<div class="row justify-content-between">
<div class="col col-auto ml-auto align-self-top">
<div class="love-icon pl-icon js-favorite-restaurant" data-category="restaurant.result" data-cooking-type="Japanese" data-country="en" data-dtm-chef="" data-dtm-city="Vancouver" data-dtm-distinction="1 star" data-dtm-district="" data-dtm-id="1214860" data-dtm-online-booking="False" data-dtm-price="none" data-dtm-region="British Columbia" data-enabled="false" data-guide="Global" data-language="en" data-pid="1214860" data-restaurant-country="ca" data-restaurant-name="Sushi Masuda" data-restaurant-selection="Canada">
<img alt="" class="love-this" src="/assets/ima

## This scrapes all 1 star restaraunts and gets name, cuisine, location, and price

In [4]:
import time
import random
from requests import get
from bs4 import BeautifulSoup
from IPython.display import clear_output
from warnings import warn

# Lists to store the scraped data
distinctions = []
names = []
locations = []
prices = []
cuisines = []

# Loop through pages (replace `pages` with the list or range of pages you want to scrape)
pages = [str(i) for i in range(1, 146)] 
start_time = time.time()
requests = 0

for page in pages:
    # Make a get request
    url = f'https://guide.michelin.com/en/restaurants/1-star-michelin/page/{page}'
    response = get(url)
    
    # Pause loop
    time.sleep(random.randint(8, 15))
    
    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print(f'Request: {requests}; Frequency: {requests/elapsed_time} requests/s')
    clear_output(wait=True)
    
    # Warning for non-200 status codes
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        break
    
    # Parse the html content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Select the container for each restaurant
    restaurants = soup.find_all('div', class_='col-md-6 col-lg-4 col-xl-3')
    
    for restaurant in restaurants:
        try:
            # Scraping distinction (assuming this page is for 1-star Michelin restaurants)
            distinction = '1 Star'
            distinctions.append(distinction)
            
            # Scraping name
            name = restaurant.find('h3', class_='card__menu-content--title').get_text(strip=True)
            names.append(name)
            
            # Scraping location
            location = restaurant.find('div', class_='card__menu-footer--score').get_text(strip=True)
            locations.append(location)
            
            # Scraping price and cuisine
            price_cuisine = restaurant.find_all('div', class_='card__menu-footer--score')
            if len(price_cuisine) > 1:
                price_cuisine_text = price_cuisine[1].get_text(strip=True)
                price = price_cuisine_text.split('·')[0].strip()  # Extract price (e.g., $$$)
                cuisine = price_cuisine_text.split('·')[-1].strip()  # Extract cuisine (e.g., French)
            else:
                price = "Not available"
                cuisine = "Not available"
            prices.append(price)
            cuisines.append(cuisine)
        
        except Exception as e:
            print(f"Error scraping restaurant: {e}")
    
# Display the data
for i in range(len(names)):
    print(f"Distinction: {distinctions[i]}, Name: {names[i]}, Location: {locations[i]}, Price: {prices[i]}, Cuisine: {cuisines[i]}")


Distinction: 1 Star, Name: Sushi Masuda, Location: Vancouver, Canada, Price: $$$$, Cuisine: Japanese
Distinction: 1 Star, Name: Okeya Kyujiro, Location: Vancouver, Canada, Price: $$$$, Cuisine: Japanese
Distinction: 1 Star, Name: Barbara, Location: Vancouver, Canada, Price: $$$$, Cuisine: Contemporary
Distinction: 1 Star, Name: iDen & QuanJuDe Beijing Duck House, Location: Vancouver, Canada, Price: $$$$, Cuisine: Chinese
Distinction: 1 Star, Name: St. Lawrence, Location: Vancouver, Canada, Price: $$$$, Cuisine: French
Distinction: 1 Star, Name: AnnaLena, Location: Vancouver, Canada, Price: $$$$, Cuisine: Contemporary
Distinction: 1 Star, Name: Published on Main, Location: Vancouver, Canada, Price: $$$, Cuisine: Contemporary
Distinction: 1 Star, Name: Kissa Tanto, Location: Vancouver, Canada, Price: $$$$, Cuisine: Fusion
Distinction: 1 Star, Name: Burdock & Co, Location: Vancouver, Canada, Price: $$$$, Cuisine: Contemporary
Distinction: 1 Star, Name: Masayoshi, Location: Vancouver, Cana

In [5]:
# Creating a DataFrame
# Create a DataFrame with the scraped data
michelin_guide = pd.DataFrame({
    'Distinction': distinctions,
    'Name': names,
    'Location': locations,
    'Price': prices,
    'Cuisine': cuisines
})



# Saving DataFrame to csv
michelin_guide.to_csv('michelin_guide_1star.csv')

michelin_guide.head()

,Distinction,Name,Location,Price,Cuisine
0,1 Star,Sushi Masuda,"Vancouver, Canada",$$$$,Japanese
1,1 Star,Okeya Kyujiro,"Vancouver, Canada",$$$$,Japanese
2,1 Star,Barbara,"Vancouver, Canada",$$$$,Contemporary
3,1 Star,iDen & QuanJuDe Beijing Duck House,"Vancouver, Canada",$$$$,Chinese
4,1 Star,St. Lawrence,"Vancouver, Canada",$$$$,French


## Again for 2 stars

In [6]:
import time
import random
from requests import get
from bs4 import BeautifulSoup
from IPython.display import clear_output
from warnings import warn

# Lists to store the scraped data
distinctions = []
names = []
locations = []
prices = []
cuisines = []

# Loop through pages (replace `pages` with the list or range of pages you want to scrape)
pages = [str(i) for i in range(1, 26)] 
start_time = time.time()
requests = 0

for page in pages:
    # Make a get request
    url = f'https://guide.michelin.com/en/restaurants/2-stars-michelin/page/{page}'
    response = get(url)
    
    # Pause loop
    time.sleep(random.randint(8, 15))
    
    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print(f'Request: {requests}; Frequency: {requests/elapsed_time} requests/s')
    clear_output(wait=True)
    
    # Warning for non-200 status codes
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        break
    
    # Parse the html content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Select the container for each restaurant
    restaurants = soup.find_all('div', class_='col-md-6 col-lg-4 col-xl-3')
    
    for restaurant in restaurants:
        try:
            # Scraping distinction (assuming this page is for 1-star Michelin restaurants)
            distinction = '2 Star'
            distinctions.append(distinction)
            
            # Scraping name
            name = restaurant.find('h3', class_='card__menu-content--title').get_text(strip=True)
            names.append(name)
            
            # Scraping location
            location = restaurant.find('div', class_='card__menu-footer--score').get_text(strip=True)
            locations.append(location)
            
            # Scraping price and cuisine
            price_cuisine = restaurant.find_all('div', class_='card__menu-footer--score')
            if len(price_cuisine) > 1:
                price_cuisine_text = price_cuisine[1].get_text(strip=True)
                price = price_cuisine_text.split('·')[0].strip()  # Extract price (e.g., $$$)
                cuisine = price_cuisine_text.split('·')[-1].strip()  # Extract cuisine (e.g., French)
            else:
                price = "Not available"
                cuisine = "Not available"
            prices.append(price)
            cuisines.append(cuisine)
        
        except Exception as e:
            print(f"Error scraping restaurant: {e}")
    
# Display the data
for i in range(len(names)):
    print(f"Distinction: {distinctions[i]}, Name: {names[i]}, Location: {locations[i]}, Price: {prices[i]}, Cuisine: {cuisines[i]}")


Distinction: 2 Star, Name: Sushi Masaki Saito, Location: Toronto, Canada, Price: $$$$, Cuisine: Japanese
Distinction: 2 Star, Name: Xin Rong Ji, Location: Chengdu, China Mainland, Price: ¥¥¥¥, Cuisine: Taizhou
Distinction: 2 Star, Name: Yu Zhi Lan, Location: Chengdu, China Mainland, Price: ¥¥¥¥, Cuisine: Sichuan
Distinction: 2 Star, Name: Molino de Urdániz, Location: Taipei, Taiwan, Price: $$$$, Cuisine: Spanish Contemporary
Distinction: 2 Star, Name: RAW, Location: Taipei, Taiwan, Price: $$$$, Cuisine: Innovative
Distinction: 2 Star, Name: logy, Location: Taipei, Taiwan, Price: $$$, Cuisine: Asian Contemporary
Distinction: 2 Star, Name: Mudan, Location: Taipei, Taiwan, Price: $$$$, Cuisine: Tempura
Distinction: 2 Star, Name: L'Atelier de Joël Robuchon, Location: Taipei, Taiwan, Price: $$$$, Cuisine: French Contemporary
Distinction: 2 Star, Name: Sons & Daughters, Location: San Francisco, USA, Price: $$$$, Cuisine: Contemporary
Distinction: 2 Star, Name: Hayato, Location: Los Angeles, 

In [7]:
# Create a DataFrame with the scraped data
michelin_guide_2 = pd.DataFrame({
    'Distinction': distinctions,
    'Name': names,
    'Location': locations,
    'Price': prices,
    'Cuisine': cuisines
})


# Saving DataFrame to csv
michelin_guide_2.to_csv('michelin_guide_2star.csv')

michelin_guide_2.head()

,Distinction,Name,Location,Price,Cuisine
0,2 Star,Sushi Masaki Saito,"Toronto, Canada",$$$$,Japanese
1,2 Star,Xin Rong Ji,"Chengdu, China Mainland",¥¥¥¥,Taizhou
2,2 Star,Yu Zhi Lan,"Chengdu, China Mainland",¥¥¥¥,Sichuan
3,2 Star,Molino de Urdániz,"Taipei, Taiwan",$$$$,Spanish Contemporary
4,2 Star,RAW,"Taipei, Taiwan",$$$$,Innovative


## Now again for 3 stars

In [8]:
import time
import random
from requests import get
from bs4 import BeautifulSoup
from IPython.display import clear_output
from warnings import warn

# Lists to store the scraped data
distinctions = []
names = []
locations = []
prices = []
cuisines = []

# Loop through pages (replace `pages` with the list or range of pages you want to scrape)
pages = [str(i) for i in range(1, 9)] 
start_time = time.time()
requests = 0

for page in pages:
    # Make a get request
    url = f'https://guide.michelin.com/en/restaurants/3-stars-michelin/page/{page}'
    response = get(url)
    
    # Pause loop
    time.sleep(random.randint(8, 15))
    
    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print(f'Request: {requests}; Frequency: {requests/elapsed_time} requests/s')
    clear_output(wait=True)
    
    # Warning for non-200 status codes
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        break
    
    # Parse the html content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Select the container for each restaurant
    restaurants = soup.find_all('div', class_='col-md-6 col-lg-4 col-xl-3')
    
    for restaurant in restaurants:
        try:
            # Scraping distinction (assuming this page is for 1-star Michelin restaurants)
            distinction = '3 Star'
            distinctions.append(distinction)
            
            # Scraping name
            name = restaurant.find('h3', class_='card__menu-content--title').get_text(strip=True)
            names.append(name)
            
            # Scraping location
            location = restaurant.find('div', class_='card__menu-footer--score').get_text(strip=True)
            locations.append(location)
            
            # Scraping price and cuisine
            price_cuisine = restaurant.find_all('div', class_='card__menu-footer--score')
            if len(price_cuisine) > 1:
                price_cuisine_text = price_cuisine[1].get_text(strip=True)
                price = price_cuisine_text.split('·')[0].strip()  # Extract price (e.g., $$$)
                cuisine = price_cuisine_text.split('·')[-1].strip()  # Extract cuisine (e.g., French)
            else:
                price = "Not available"
                cuisine = "Not available"
            prices.append(price)
            cuisines.append(cuisine)
        
        except Exception as e:
            print(f"Error scraping restaurant: {e}")
    
# Display the data
for i in range(len(names)):
    print(f"Distinction: {distinctions[i]}, Name: {names[i]}, Location: {locations[i]}, Price: {prices[i]}, Cuisine: {cuisines[i]}")


Distinction: 3 Star, Name: Taïrroir, Location: Taipei, Taiwan, Price: $$$$, Cuisine: Taiwanese contemporary
Distinction: 3 Star, Name: JL Studio, Location: Taichung, Taiwan, Price: $$$$, Cuisine: Singaporean
Distinction: 3 Star, Name: Le Palais, Location: Taipei, Taiwan, Price: $$$$, Cuisine: Cantonese
Distinction: 3 Star, Name: Addison, Location: San Diego, USA, Price: $$$$, Cuisine: Contemporary
Distinction: 3 Star, Name: Atelier Crenn, Location: San Francisco, USA, Price: $$$$, Cuisine: Contemporary
Distinction: 3 Star, Name: The French Laundry, Location: Yountville, USA, Price: $$$$, Cuisine: Contemporary
Distinction: 3 Star, Name: Quince, Location: San Francisco, USA, Price: $$$$, Cuisine: Contemporary
Distinction: 3 Star, Name: SingleThread, Location: Healdsburg, USA, Price: $$$$, Cuisine: Contemporary
Distinction: 3 Star, Name: Benu, Location: San Francisco, USA, Price: $$$$, Cuisine: Asian
Distinction: 3 Star, Name: Zén, Location: Singapore, Price: $$$$, Cuisine: European Conte

In [9]:
# Create a DataFrame with the scraped data
michelin_guide_3 = pd.DataFrame({
    'Distinction': distinctions,
    'Name': names,
    'Location': locations,
    'Price': prices,
    'Cuisine': cuisines
})


# Saving DataFrame to csv
michelin_guide_3.to_csv('michelin_guide_3star.csv')

michelin_guide_3.head()

,Distinction,Name,Location,Price,Cuisine
0,3 Star,Taïrroir,"Taipei, Taiwan",$$$$,Taiwanese contemporary
1,3 Star,JL Studio,"Taichung, Taiwan",$$$$,Singaporean
2,3 Star,Le Palais,"Taipei, Taiwan",$$$$,Cantonese
3,3 Star,Addison,"San Diego, USA",$$$$,Contemporary
4,3 Star,Atelier Crenn,"San Francisco, USA",$$$$,Contemporary


## Now want to merge the 3 and remove any ones not from the USA

In [20]:
import pandas as pd

# Load the CSV files
df_1star = pd.read_csv('michelin_guide_1star.csv')
df_2star = pd.read_csv('michelin_guide_2star.csv')
df_3star = pd.read_csv('michelin_guide_3star.csv')

# Concatenate the DataFrames
combined_df = pd.concat([df_1star, df_2star, df_3star], ignore_index=True)

# Filter only restaurants from the USA
usa_restaurants = combined_df[combined_df['Location'].str.contains('USA|United States', case=False, na=False)]
usa_restaurants = usa_restaurants[usa_restaurants['Price'].str.contains('\$', na=False)]
usa_restaurants['Price_Numerical'] = usa_restaurants['Price'].replace(['$', '$$', '$$$', '$$$$'], ['1', '2', '3', '4'])

# Save the filtered DataFrame to a new CSV
usa_restaurants.to_csv('michelin_restaurants_usa.csv', index=False)

print(f"Successfully combined and filtered restaurants. Saved to 'michelin_restaurants_usa.csv'.")


Successfully combined and filtered restaurants. Saved to 'michelin_restaurants_usa.csv'.


<>:13: SyntaxWarning: invalid escape sequence '\$'
<>:13: SyntaxWarning: invalid escape sequence '\$'
C:\Users\shaun\AppData\Local\Temp\ipykernel_33384\1204325935.py:13: SyntaxWarning: invalid escape sequence '\$'
  usa_restaurants = usa_restaurants[usa_restaurants['Price'].str.contains('\$', na=False)]
